# 1. Initializations

## 1.1 General imports

In [ ]:
### Data management
import pandas as pd
import numpy as np
import random

### Machine Learning

# transformation
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, OneHotEncoder

# models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# resampling
from imblearn.over_sampling import SMOTE

# metrics and evaluation
from sklearn.metrics import f1_score, confusion_matrix
from scipy.stats import chi2_contingency, probplot
from xgboost import plot_importance

### Data Viz

# graphical basics
import matplotlib.pyplot as plt
%matplotlib inline

# graphical seaborn
import seaborn as sns

# # graphical plotly
# import plotly.graph_objects as go
# import plotly.express as px
# # for jupyter notebook display management
# import plotly.io as pio
# pio.renderers.default = "notebook"


## 1.2 General dataframe functions

In [ ]:
import smartcheck.dataframe_common as dfc

## 1.3 General Classification functions

In [ ]:
import smartcheck.classification_common as cls

# 2. Loading and Data Quality

## 2.1 Loading of data sets and general exploration

### 2.1.1 VELO COMPTAGE (Main Data Set)

#### Loading and column management (columns names normalization)

In [ ]:
df_cpt_velo_raw = dfc.load_dataset_from_config('velo_comptage_data', sep=';')

if df_cpt_velo_raw is not None and isinstance(df_cpt_velo_raw, pd.DataFrame):
    display(df_cpt_velo_raw.head())
    dfc.log_general_info(df_cpt_velo_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_cpt_velo_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_cpt_velo_raw))
    df_cpt_velo = dfc.normalize_column_names(df_cpt_velo_raw)

#### Global description and correlation

In [ ]:
df_cpt_velo.info()
display(df_cpt_velo.head())
df_cpt_velo_desc = df_cpt_velo.select_dtypes(include=np.number).describe()
display(df_cpt_velo_desc)
df_cpt_velo_cr = df_cpt_velo.select_dtypes(include=np.number).corr()
display(df_cpt_velo_cr)

### 2.1.2 VELIB DISPO (Optional Dataset)

#### Loading and column management (columns names normalization)

In [ ]:
df_disp_velib_raw = dfc.load_dataset_from_config('velib_dispo_data', sep=';')

if df_disp_velib_raw is not None and isinstance(df_disp_velib_raw, pd.DataFrame):
    display(df_disp_velib_raw.head())
    dfc.log_general_info(df_disp_velib_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_disp_velib_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_disp_velib_raw))
    df_disp_velib = dfc.normalize_column_names(df_disp_velib_raw)

#### Global description and correlation

In [ ]:
df_disp_velib.info()
display(df_disp_velib.head())
df_cpt_velo_desc = df_disp_velib.select_dtypes(include=np.number).describe()
display(df_cpt_velo_desc)
df_cpt_velo_cr = df_disp_velib.select_dtypes(include=np.number).corr()
display(df_cpt_velo_cr)
dfc.display_variable_info(df_disp_velib, 5)

#### Cross Distribution inspection

In [ ]:
# Analyse de la distribution d'une variable spécique en relation avec les autres de son dataframe
ref_col = 'station_en_fonctionnement' # ici notre variable cible
dfc.analyze_by_reference_variable(df_disp_velib, ref_col)

In [ ]:
# Analyse croisée de la distribution d'une variable spécifique en fonction d'autres variables (quantitatives ou qualitatives) du dataframe
ref_col = 'station_en_fonctionnement' # ici notre variable cible
cross_columns = [ref_col] + ['borne_de_paiement_disponible', 'retour_velib_possible']
dfc.log_cross_distributions(
    df_disp_velib[cross_columns], 
    ref_col
)

In [ ]:
# Analyse croisée d'une variable en fonction d'une autre
ref_col = 'borne_de_paiement_disponible'
target_col = 'station_en_fonctionnement'
display(pd.crosstab(df_disp_velib[ref_col], df_disp_velib[target_col]))

In [ ]:
# Analyse croisée des d'une variable en fonction d'une autre (en conservant les NaN)
ref_col = 'station_opening_hours'
target_col = 'station_en_fonctionnement'
ref_cross_tab = pd.crosstab(df_disp_velib[ref_col], df_disp_velib[target_col], dropna=False, normalize=True)
display(ref_cross_tab)

# Catégorisation et normalisation
ref_col_val_norm = np.where(
    df_disp_velib[ref_col].isin(ref_cross_tab[ref_cross_tab['OUI'] >= 0.8].index.tolist()), 
    1, 
    0
)
df_disp_velib[ref_col] = ref_col_val_norm
ref_cross_tab_norm = pd.crosstab(df_disp_velib[ref_col], df_disp_velib[target_col], dropna=False, normalize=True)
display(ref_cross_tab_norm)

#### Signifiance against target evaluation

In [ ]:
# Verification de la signifiance des variables explicatives par rapport à une variable cible
ref_col = 'retour_velib_possible'
target_col = 'station_en_fonctionnement'
# Génération des colonnes dummies pour ref_col
ref_col_dummies = pd.get_dummies(df_disp_velib[ref_col], prefix=ref_col)
print("Colonnes dummies générées :", list(ref_col_dummies.columns))
# Pour chaque modalité de cette variable (dummy 0/1), tester sa signifiance avec la variable cible
for col in ref_col_dummies:
    # Test du Chi-Deux
    cross_tab = pd.crosstab(df_disp_velib[target_col], ref_col_dummies[col])
    if cross_tab.shape[1] != 2:
        print(f"⚠️ Modalité [{col}] ignorée (1 seule valeur présente)")
        continue
    stat, p, _, _ = chi2_contingency(cross_tab)
    # V de Cramer
    V_Cramer = np.sqrt(
        stat/cross_tab.values.sum())
    # On affiche uniquement les variables significatives et dont le V de Cramer est supérieur à 0.1
    # Faible : Valeur autour de 0.1 ;
    # Moyenne : Valeur autour de 0.3 ;
    # Elevée : Valeur autour et supérieure à 0.5.
    # Lorsque la valeur du V de Cramer est très élevée (aux alentours de 0.8 et plus), on soupçonne généralement de la multicolinéarité.
    result = 'significative' if (p < 0.05) and (V_Cramer > 0.1) else 'NON signficative'
    print(f"Variable [{col}] {result} Vs [{target_col}]: p-value[{p:.5f}], V_Cramer[{V_Cramer:.5f}]")

## 2.2 Data quality refinement

### 2.2.1 VELO COMPTAGE (Main Dataset)

In [ ]:
# Original backup and duplicates management
df_cpt_velib_orig = df_cpt_velo.copy()
df_cpt_velo = df_cpt_velo.drop_duplicates()

### 2.2.2 VELIB DISPO (Optional Dataset)

In [ ]:
# Original backup and duplicates management
df_disp_velib_orig = df_disp_velib.copy()
df_disp_velib = df_disp_velib.drop_duplicates()

In [ ]:
# transformation manuelle de la variable cible 
# /!\ attention /!\ les transformation d'encodage meme les plus simples ne sont pas sans fuite de données car elles changent la structure
# exemple : un get_dummies qui va créer une colonne sur une valeur rare influence par les colonnes additionnelles (creation de sub features)
df_disp_velib.station_en_fonctionnement = df_disp_velib.station_en_fonctionnement.apply(lambda x: 1 if x=='OUI' else 0).astype(int)

# 3. Data Viz' and Analysis

## 3.1 General Data Viz'

In [ ]:
# Vérificationn graphique de la répartition en loi normale de chaque données numérique
for col in df_disp_velib.select_dtypes(include='number').columns:
    probplot(df_disp_velib[col], dist="norm", plot=plt)
    plt.suptitle(f"Column {col}")
    plt.show()

## 3.2 Quantitative mono variable distribution

## 3.3 Qualitative mono variable distribution

## 3.4 Qualitative multi variable distribution

## 3.5 Quantitative multi variable correlation

# 4. Division in Train/Test

In [ ]:
# df_disp_velib = df_disp_velib_orig.copy()

In [ ]:
# (temporaire) Ajustement : enlever les variables non retravaillées pour le moment
df_disp_velib = df_disp_velib.drop(columns=['identifiant_station',
                                            'nom_station', 
                                            'actualisation_de_la_donnee', 
                                            'coordonnees_geographiques', 
                                            'nom_communes_equipees',
                                            'code_insee_communes_equipees',
                                            'station_opening_hours'])

In [ ]:
# Separation features (X) et target (y) pour train et test
target_col = 'station_en_fonctionnement'
features = df_disp_velib.drop(target_col, axis=1)
target = df_disp_velib[target_col]
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=66)
print("Train Set (X/y):", X_train.shape, y_train.shape)
print("Test Set (X/y):", X_test.shape, y_test.shape)

# 5. Feature engineering
Règle d'or : Toute opération qui "apprend" des données (i.e. utilise l’ensemble des valeurs pour calculer quelque chose) doit être faite après le split train/test — c’est-à-dire uniquement sur le train.

| Type de transformation                                                                                    | À faire avant le split ?                    | Détails                                                            |
| --------------------------------------------------------------------------------------------------------- | ------------------------------------------- | ------------------------------------------------------------------ |
| ✅ Création de features basées sur les colonnes existantes (ex: `BMI = weight / height²`)                  | **Avant**                                   | Pas de risque de fuite car c’est purement déterministe.            |
| ⚠️ Calculs dépendant de la distribution (moyennes, encodage fréquentiel, imputation par la médiane, etc.) | **Après** (sur le train uniquement)         | Risque de fuite de données si appliqué sur l’ensemble avant split. |
| ✅ Ajout de features exogènes fixes (données météo, géographiques, calendaires, etc.)                      | **Avant**                                   | Pas de dépendance au `target` ni à la répartition train/test.      |
| ⚠️ Encoding (`LabelEncoder`, `OneHot`, `TargetEncoding`, etc.)                                            | **Fit sur train, transform sur train/test** | Toujours fitter uniquement sur le `train`.                         |
| ⚠️ Standardisation / normalisation (Scaler)                                                               | **Fit sur train, transform sur train/test** | Pareil : `.fit()` sur train, `.transform()` sur test.              |


## 5.1 Modification localisées sur les variables d'entrainement

In [ ]:
# Exemple de modification localisée en fonction de la proximité à la médiane d'autre variables
# mask = (
#     (train['Gender'].isna()) &
#     (abs(train['Age'] - 30) > abs(train['Age'] - 41)) & # L’âge est plus proche de 41 que de 30
#     (train['Previously_Insured'] == 0) & # La personne n’était pas assurée auparavant
#     (train['Vehicle_Damage'] == 1) # Elle a subi un dommage sur son véhicule
# )
# train.loc[mask, 'Gender'] = 0

In [ ]:
# Exemple de modification par répartition spécifique entre deux valeurs 0 et 1
# proportion_tab = [0] * 55 + [1] * 45
# mask = (
#     (train['Gender'].isna()) &
# )
# train.loc[mask, 'Gender'] = train.loc[mask, 'Gender'].apply(lambda x: random.choice(proportion))

## 5.2 Preprocessing

### 5.2.1 Scaling (données quantitatives)

In [ ]:
# - ni outlier ni distribution loi normale : min/max
# - sans outlier mais distribution loi normale : standard
# - avec outlier : Robust 
mm_scal = MinMaxScaler()
r_scal = RobustScaler()
s_scal = StandardScaler()

r_scal_col = ['velos_mecaniques_disponibles', 'nombre_total_velos_disponibles']
df_disp_velib[r_scal_col] = s_scal.fit_transform(df_disp_velib[r_scal_col])

### 5.2.1 Encoding (données qualitatives)

In [ ]:
# Technique                 Type                Colonnes créées     Principe
# get_dummies()	            Nominale	        N (ou N–1)	        Binaire par modalité
# OneHotEncoder	            Nominale, Cyclique	N	                Colonne 0/1 par modalité
# Sum Encoding	            Nominale	        N–1	                Différence avec moyenne globale
# Helmert Encoding	        Nominale	        N–1	                Contraste avec moyenne des modalités précédentes
# Backward Difference	    Ordinale	        N–1	                Contraste avec moyenne des modalités suivantes
# Binary Encoding	        Nominale	        log₂(N)	            Encodage binaire de l’index
# Hashing Encoding	        Nominale	        n_components	    Hash des modalités sur colonnes fixes
# Label Encoding	        Ordinale	        1	                Entier arbitraire
# Ordinal Encoding	        Ordinale	        1	                Rang croissant des modalités
# Target Encoding	        Nominale/Ordinale	1	                Moyenne de la cible par modalité
# Mean Encoding	            Nominale/Ordinale	1	                Idem Target Encoding
# Frequency Encoding	    Nominale/Ordinale	1	                Fréquence d'apparition
# Leave-One-Out	            Nominale/Ordinale	1	                Moyenne de la cible, sauf ligne courante
# James-Stein Encoding	    Nominale/Ordinale	1	                Moyenne pondérée par variance intercatégorie
# M-Estimate Encoding	    Nominale/Ordinale	1	                Moyenne cible lissée vers moyenne globale
# Probability Ratio	        Ordinale, binaire	1	                Log du ratio de probas classe 1 / classe 0
# WOE Encoding	            Ordinale, binaire	1	                Log( %positif / %négatif )
# Thermometer Encoding	    Ordinale	        N	                1 si la modalité est ≤ à une valeur
# Trigonométrique (sin/cos)	Cyclique	        2	                Encode la cyclicité
# Fourier / Radial	        Cyclique	        Variable	        Approximation périodique (base)
ohe_enc_col = ['borne_de_paiement_disponible', 'retour_velib_possible']
ohe_enc = OneHotEncoder(handle_unknown='ignore', sparse_output = False)
# Appliquer OneHotEncoder
X_train_enc_cat = ohe_enc.fit_transform(X_train[ohe_enc_col])
X_test_enc_cat = ohe_enc.transform(X_test[ohe_enc_col])
# Ajout des colonnes encodées à un DataFrame car le resultat de enc.fit/transform estun ndarray sans index/colonnes
X_train_cat_df = pd.DataFrame(X_train_enc_cat, columns=ohe_enc.get_feature_names_out(ohe_enc_col), index=X_train.index)
X_test_cat_df = pd.DataFrame(X_test_enc_cat, columns=ohe_enc.get_feature_names_out(ohe_enc_col), index=X_test.index)
# Suppression des colonnes catégoriques originales et ajout des colonnes encodées
X_train = pd.concat([X_train.drop(columns=ohe_enc_col), X_train_cat_df], axis=1)
X_test = pd.concat([X_test.drop(columns=ohe_enc_col), X_test_cat_df], axis=1)

# 6. Selection and Model Training

### 6.1 Quick Logistic Regression

In [ ]:
# Logistic Regression configuration
logit_config = {
    "target": "station_en_fonctionnement",
    "features": ['nombre_bornettes_libres', 'nombre_total_velos_disponibles']
}

# Coefficient adjustment configuration
adjustment_config = {
    "nombre_bornettes_libres": {
        "type": "normalize",
        "range": (0, 68)
    },
    "nombre_total_velos_disponibles": {
        "type": "normalize",
        "range": (0, 65)
    },
    # "nombre_total_velos_disponibles": {
    #     "type": "inverse"
    # },
}
df_train = pd.concat([X_train, y_train], axis=1)
subset = logit_config["features"]+[logit_config["target"]]
cls.logit_analysis(df_train[subset], logit_config, adjustment_config)

### 6.2 SMOTE/Under/Over Sampling (if necessary)

In [ ]:
cls.cross_validation_with_resampling(X_train, y_train, LogisticRegression())

In [ ]:
cls.cross_validation_with_resampling(X_train, y_train, XGBClassifier(eval_metric="error"))

### 6.3 Adding threashold (if necessary)

In [ ]:
cls.cross_validation_with_resampling_and_threshold(X_train, y_train, LogisticRegression())

### 6.4 Switching to other models (if necessary)

In [ ]:
cls.cross_validation_with_resampling_and_threshold(X_train, y_train, XGBClassifier(eval_metric="error"))

# 7. Best model application and visualizing results

In [ ]:
# Resampling sur la base d'entraînement
smote = SMOTE()
X_train_b_smote, y_train_b_smote = smote.fit_resample(X_train, y_train)

# Définition et entraînement du modèle
clf_XGB_opti = XGBClassifier(eval_metric="error")
clf_XGB_opti.fit(X_train_b_smote, y_train_b_smote)

# Prédiction du modèle (Seuil précédemment établi)
preds = np.where(clf_XGB_opti.predict_proba(X_test)[:, 1] > 0.026, 1, 0)

# Evaluation
print("Modèle avec rééchantillonnage et optimisation du seuil")
print("F1-Score : ", f1_score(preds, y_test))
print(confusion_matrix(preds, y_test))

# Modèle de base
print("Modèle de base")
clf_XGB_base = XGBClassifier(eval_metric="error")
clf_XGB_base.fit(X_train, y_train)
print("F1-Score : ", f1_score(clf_XGB_base.predict(X_test), y_test))
print(confusion_matrix(clf_XGB_base.predict(X_test), y_test))
plot_importance(clf_XGB_base) 
plt.show()